In [10]:
import win32com.client
import pandas as pd

In [11]:

# Path to the template file
HDD_TEMPLATE_FILE_PATH = "Macrogen_EU_CI_template_202001.lbx"
USB_TEMPLATE_FILE_PATH = "Macrogen_EU_CI_template_USB_202001.lbx"

# Printer name
PRINTER_NAME = "Brother QL-820NWB"

# Create a b-PAC client
client = win32com.client.DispatchEx("bpac.Document")

# Open the template file
# client.Open(TEMPLATE_FILE_PATH)


In [13]:
df_worksheet = pd.read_excel('data/Backup_orders.xlsx')
filtered_rows = df_worksheet[df_worksheet['ANALYZER'] == 'Changlin Ke'].fillna('')

In [14]:
filtered_rows = filtered_rows[filtered_rows['ORDER #'].isin(['EN00001684', 'EN00000357', 'EN00001931', 'EN00001298'])]

In [15]:
filtered_rows

,ORDER #,CUSTOMER,PRE NAME,POST NAME,APPLICATION,SPECIES,REPORT ARRIVAL DATE,ANALYZER,PROGRESS,Mounted,HDD Type,Delivered,Time,Unnamed: 13
20,EN00001298,Leen Dierick,Shifan Ding,Chih Lin,Whole Genome Resequencing,Other,2023-03-20,Changlin Ke,,,USB(256GB),NaT,2023-03-20 17:15:40,N
28,EN00000357,Monika Madrova,Christopher,Santiago Parga,Exome Sequencing,Homo sapiens(human),2022-10-12,Changlin Ke,,,HDD(1TB),NaT,2022-10-12 11:47:39,N
29,EN00001931,Pieter Asselman,Shifan Ding,Chih Lin,Whole Genome De novo Sequencing,Other,2023-04-03,Changlin Ke,,,USB(256GB),NaT,2023-04-03 19:23:28,N
31,EN00001684,Taeksun Song,Mary Woonah Seo,Anthea Grum,Whole Genome Resequencing,Other,2023-03-27,Changlin Ke,,,HDD(1TB),NaT,2023-03-27 10:44:23,N


In [16]:
for _, row in filtered_rows.iterrows():
# Replace text fields in the template
# The 'SetField' method requires two arguments: the field name and the new text
# Replace 'FIELD_NAME' with the actual field name in your template, and 'NEW_TEXT' with the new text you want to insert
# Open the template file
    if 'HDD' in row['HDD Type']:
        client.Open(HDD_TEMPLATE_FILE_PATH)
        
        # Replace text fields in the template based on the columns in the Excel file
        # Adjust the column names as needed based on your Excel sheet structure
        client.GetObject("CustomerName").Text = row['CUSTOMER']
        client.GetObject("Order").Text = row['ORDER #']
        client.GetObject("FileType").Text = 'Raw Data Backup' # to be changed
        client.GetObject("DiskNo.").Text = row['HDD Type'] # to be changed
        # Print the label twice
        i = 0
        while i < 2:
            if not client.DoPrint(0, PRINTER_NAME):
                print(f"Print failed for row {row.name} with error code:", client.ErrorCode)
            i += 1
        
    elif 'USB' in (row['HDD Type']):
        client.Open(USB_TEMPLATE_FILE_PATH)

        client.GetObject("CustomerName_big").Text = row['CUSTOMER']
        client.GetObject("Orderbig").Text = row['ORDER #']
        client.GetObject("CustomerName_small").Text = row['CUSTOMER']
        client.GetObject("OrderSmall").Text = row['ORDER #']
        # Print the label
        if not client.DoPrint(0, PRINTER_NAME):
            print(f"Print failed for row {row.name} with error code:", client.ErrorCode)
    else:
        continue

    

In [8]:
# Print the label
# The 'DoPrint' method requires two arguments: the print option (set to 0 for default) and the printer name
if not client.DoPrint(0, PRINTER_NAME):
    print("Print failed with error code:", client.ErrorCode)